In [36]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

In [2]:
## 데이터 전처리 ##
def fillna(df):
    df['Age'].fillna(titanic_df['Age'].mean(), inplace = True)
    df['Cabin'].fillna('N', inplace = True)
    df['Embarked'].fillna('N', inplace = True)
    df['Fare'].fillna(0, inplace=True)
    return df    
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df
def format_features(df):
    df['Cabin']=df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [30]:
## 데이터 x,y분할 ##
titanic_df = pd.read_csv('./titanic_train.csv')
y=titanic_df['Survived']
x=titanic_df.drop('Survived', axis=1)
x=transform_features(x)

In [28]:
## DT, RF, LR ##
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression()
dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
print('DT 정확도:{0:.4f}'.format(accuracy_score(y_test,dt_pred)))
rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
print('RF 정확도:{0:.4f}'.format(accuracy_score(y_test,rf_pred)))
lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
print('LR 정확도:{0:.4f}'.format(accuracy_score(y_test,lr_pred)))

DT 정확도:0.7877
RF 정확도:0.8547
LR 정확도:0.8492


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
## Kfold Class ##
def exec_kfold(clf, folds=5):
    kfold = KFold(n_splits=folds)
    scores=[]
    for iter_count, (train_index, test_index) in enumerate(kfold.split(x)):
        x_train, x_test=x.values[train_index],x.values[test_index]
        y_train, y_test=y.values[train_index],y.values[test_index]
        clf.fit(x_train, y_train)
        predictions=clf.predict(x_test)
        accuracy=accuracy_score(y_test,predictions)
        scores.append(accuracy)
        print('교차검증{0}정확도:{1:.4f}'.format(iter_count,accuracy))
    mean_score=np.mean(scores)
    print('평균정확도:{0:.4f}'.format(mean_score))
dt_clf = DecisionTreeClassifier(random_state=11)
exec_kfold(dt_clf, folds=5)

교차검증0정확도:0.7542
교차검증1정확도:0.7809
교차검증2정확도:0.7865
교차검증3정확도:0.7697
교차검증4정확도:0.8202
평균정확도:0.7823


In [32]:
## cross_val_score() ##
scores = cross_val_score(dt_clf, x, y, cv=5)
for iter_count, accuracy in enumerate(scores):
    print('교차검증{0}정확도:{1:.4f}'.format(iter_count,accuracy))
print('평균정확도:{0:.4f}'.format(np.mean(scores)))

교차검증0정확도:0.7430
교차검증1정확도:0.7753
교차검증2정확도:0.7921
교차검증3정확도:0.7865
교차검증4정확도:0.8427
평균정확도:0.7879


In [34]:
##GridSearchCV class ##
parameters={'max_depth':[2,3,5,10],'min_samples_split':[2,3,5]
          ,'min_samples_leaf':[1,5,8]}
grid_dclf=GridSearchCV(dt_clf,param_grid=parameters,scoring='accuracy',cv=5)
grid_dclf.fit(x_train,y_train)


In [39]:
export_graphviz(dt_clf, out_file='tree.dot', class_names=titanic_df.target_names, 
                feature_names=titanic_df.feature_names, impurity=True, filled=True)
with open('tree.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

AttributeError: 'DataFrame' object has no attribute 'target_names'